In [2]:

import pandas as pd
CONFIG_1 = {
    'user': 'neo_reports',
    'password': 'gh2uyti56hgk2h',
    'host': '92.38.186.22',
    'port': '3306',
    'raise_on_warnings': True
}
import warnings
import mysql.connector
connection = mysql.connector.connect(**CONFIG_1)
import datetime
warnings.filterwarnings("ignore")


In [17]:
with open('query_new\Балансы и эквити 2 запроса нео эктив\Active\Active балансы.txt', mode='r', encoding='utf-8')  as f:
    data = f.read()

df = pd.read_sql_query(data, con=connection)


with open('query_new\Балансы и эквити 2 запроса нео эктив\Active\Active эквити.txt', mode='r', encoding='utf-8')  as f:
    data = f.read()

df_2 = pd.read_sql_query(data, con=connection)
df_2

,Date,Start,Equity,BBOOK,ABOOK
0,2025-05-05,0,57290.25,57290.25,0.0


In [ ]:
from tabulate import tabulate
import telebot
import datetime

def send_summary_to_telegram(df, df_2, company=""):
    """
    Формирует и возвращает сводку по Balance и Equity по блокам: Total, ABOOK, BBOOK.
    df — Balance, df_2 — Equity.
    """

    def format_number(n):
        return f"{int(n):,}".replace(",", "’") + " USD"

    def build_block(title, balance, equity):
        return (
            f'''\n🔹 *{title}*
Balance: {format_number(balance)}
Equity:  {format_number(equity)}'''
        )

    # Извлечение даты
    date = datetime.datetime.strftime(df.Date.iloc[0], '%d-%m-%Y')

    # Генерация блоков
    block_1 = build_block(
        f"Total ({company})",
        df.Balance.iloc[0],
        df_2.Equity.iloc[0]
    )

    block_2 = build_block(
        f"ABOOK ({company})",
        df.ABOOK.iloc[0],
        df_2.ABOOK.iloc[0]
    )

    block_3 = build_block(
        f"BBOOK ({company})",
        df.BBOOK.iloc[0],
        df_2.BBOOK.iloc[0]
    )

    # Сборка сообщения
    caption = f"*#PNL&Equity📊*\n📅 {date}"
    summary = f"{caption}{block_1}{block_2}{block_3}"

    # Усечение по лимиту Telegram
    if len(summary) > 4000:
        summary = summary[:3990] + "\n..."

    return summary


In [82]:
from tabulate import tabulate
import telebot

# 05-05-2025              Balance               Equity 
# Total             3’210’363 USD        3’105’176 USD 
# ABOOK             2’862’966 USD        2’805’253 USD 
# BBOOK               347’396 USD          299’923 USD



def send_summary_to_telegram(df, df_2, caption=None):
    """
    Формирует и отправляет сводку в одну строку в Telegram:
    Balance и Equity с разбивкой на ABOOK/BBOOK.
    df - balance
    df_2 - Equity
    """

    def format_number(n):
        return f"{int(n):,}".replace(",", "’") + " USD"

    # Извлекаем значения
    total_balance = format_number(df.Balance.iloc[0])
    abook_balance = format_number(df.ABOOK.iloc[0])
    bbook_balance = format_number(df.BBOOK.iloc[0])

    total_equity = format_number(df_2.Equity.iloc[0])
    abook_equity = format_number(df_2.ABOOK.iloc[0])
    bbook_equity = format_number(df_2.BBOOK.iloc[0])
    date = datetime.datetime.strftime(df.Date[0], '%d-%m-%Y')

    summary = f'''
#Баланс
Дата: {date}
Total: {total_balance} 
ABOOK: {abook_balance}
BBOOK: {bbook_balance}

#Equity
Total: {total_equity} 
ABOOK: {abook_equity} 
BBOOK: {bbook_equity} 
'''
    if caption:
        summary = f"*{caption}*\n{summary}"

    return summary


In [83]:
def read_sql(path: str):
    with open(path, mode='r', encoding='utf-8')  as f:
        data = f.read()
    return  pd.read_sql_query(data, con=connection)



def balance_and_E(broker: str):
    'Принимает на вход название Брокера(NEO, EKTIV) и возвращает готовую строку'
    if broker == 'NEO':
        df_balance = read_sql('query_new\Балансы и эквити 2 запроса нео эктив\НЕО\НЕО Балансы.txt')
        df_eq = read_sql('query_new\Балансы и эквити 2 запроса нео эктив\НЕО\НЕО Эквити.txt')
        return send_summary_to_telegram(df=df_balance, df_2=df_eq, caption='#PNL&Equity📊')
    else:
        df_balance = read_sql('query_new\Балансы и эквити 2 запроса нео эктив\Active\Active балансы.txt')
        df_eq = read_sql('query_new\Балансы и эквити 2 запроса нео эктив\Active\Active эквити.txt')
        return send_summary_to_telegram(df=df_balance, df_2=df_eq, caption='#Balance&Equity📊')

print(balance_and_E('NEO'))

*#PNL&Equity📊*

#Баланс
Дата: 05-05-2025
Total: 3’210’363 USD 
ABOOK: 2’862’966 USD
BBOOK: 347’396 USD

#Equity
Total: 3’105’176 USD 
ABOOK: 2’805’253 USD 
BBOOK: 299’923 USD 



# PNL

In [84]:
def PNL_get(broker: str):
    'Принимает на вход название Брокера (NEO, EKTIV) и возвращает готовую однострочную строку PNL'
    # Neo MU или Active Broker
    if broker == 'NEO':
        company = 'Neo MU'
        df_1 = read_sql(r'query_new\ПНЛ показатели нео эктив\NEO\NEO АБУК + ББУК часть без партнёров.txt')
        df_2 = read_sql(r'query_new\ПНЛ показатели нео эктив\NEO\NEO АБУК часть.txt')
        df_3 = read_sql(r'query_new\ПНЛ показатели нео эктив\NEO\NEO ББУК часть.txt')
    else:
        company = 'Active Broker'
        df_1 = read_sql(r'query_new\ПНЛ показатели нео эктив\Active\Active АБУК + ББУК часть без партнёров.txt')
        df_2 = read_sql(r'query_new\ПНЛ показатели нео эктив\Active\Active АБУК часть.txt')
        df_3 = read_sql(r'query_new\ПНЛ показатели нео эктив\Active\Active ББУК часть.txt')

    def format_number(n):
        return f"{round(n):,}".replace(",", "’") + " USD"

    # Вспомогательная функция для генерации блока
    def build_block(title, today, yesterday, curr_week, past_week, curr_month):
        return (
            f'''\n🔹 *{title}* 
Сегодня: {format_number(today)} 
Вчера: {format_number(yesterday)}
Тек. неделя: {format_number(curr_week)}
Пред. неделя: {format_number(past_week)}
Тек. месяц: {format_number(curr_month)}''')
        

    # Генерация блоков
    block_1 = build_block(
        f"ABOOK + BBOOK PL {company} (без выплат партнёров)",
        df_1['PL_kompani_Bbok_Abook_Today'].iloc[0],
        df_1['PL_kompani_Bbok_Abook_YESTERDAY'].iloc[0],
        df_1['PL_kompani_Bbok_Abook_WEEK'].iloc[0],
        df_1['PL_kompani_Bbok_Abook_past_WEEK'].iloc[0],
        df_1['PL_kompani_Bbok_Abook_MONTH'].iloc[0]
    )

    block_2 = build_block(
        f"ABOOK PL {company}",
        df_2['Today'].iloc[0],
        df_2['Yesterday'].iloc[0],
        df_2['Current_Week'].iloc[0],
        df_2['past_Week'].iloc[0],
        df_2['Current_Month'].iloc[0]
    )

    block_3 = build_block(
        f"BBOOK PL {company}",
        df_3['PL_kompani_Bbok_Today'].iloc[0],
        df_3['PL_kompani_Bbok_YESTERDAY'].iloc[0],
        df_3['PL_kompani_Bbok_WEEK'].iloc[0],
        df_3['PL_kompani_past_WEEK'].iloc[0],
        df_3['PL_kompani_Bbok_MONTH'].iloc[0]
    )

    # Сборка в одну строку (Telegram-friendly)
    caption = "*#PNL📈*"
    summary = f"{caption}{block_1}\n{block_2}\n{block_3}"

    # Усечём, если больше 4000 символов
    if len(summary) > 4000:
        summary = summary[:3990] + "\n..."

    return summary
print(PNL_get('fdf'))

*#PNL📈*
🔹 *ABOOK + BBOOK PL Active Broker (без выплат партнёров)* 
Сегодня: 3’808 USD 
Вчера: 0 USD
Тек. неделя: 3’808 USD
Пред. неделя: 2’342 USD
Тек. месяц: 7’901 USD

🔹 *ABOOK PL Active Broker* 
Сегодня: 0 USD 
Вчера: 0 USD
Тек. неделя: 0 USD
Пред. неделя: 0 USD
Тек. месяц: 0 USD

🔹 *BBOOK PL Active Broker* 
Сегодня: 3’808 USD 
Вчера: 0 USD
Тек. неделя: 3’808 USD
Пред. неделя: 2’342 USD
Тек. месяц: 7’901 USD


In [76]:
from tabulate import tabulate
import telebot

# Укажи свой токен и chat_id
TELEGRAM_TOKEN = '7840597904:AAHrwP2adJsCLSg3myhTDEW-eJ4gtujG-UU'
CHAT_ID = "620211681"



bot = telebot.TeleBot(TELEGRAM_TOKEN)

bot.send_message(chat_id=CHAT_ID, text=res, parse_mode='Markdown')

def send_summary_to_telegram(df, df_2, caption=None):
    """
    Формирует и отправляет сводку в одну строку в Telegram:
    Balance и Equity с разбивкой на ABOOK/BBOOK.
    """

    def format_number(n):
        return f"{int(n):,}".replace(",", "’") + " USD"

    # Извлекаем значения
    total_balance = format_number(df.Balance.iloc[0])
    abook_balance = format_number(df.ABOOK.iloc[0])
    bbook_balance = format_number(df.BBOOK.iloc[0])

    total_equity = format_number(df_2.Equity.iloc[0])
    abook_equity = format_number(df_2.ABOOK.iloc[0])
    bbook_equity = format_number(df_2.BBOOK.iloc[0])
    date = datetime.datetime.strftime(df.Date[0], '%d-%m-%Y')
    # Печатаем красиво в виде двух колонок
    s1 = (f"{date:<10} {'Balance':>20} {'Equity':>20} \n")
    s2 =(f"{'Total':<10} {total_balance:>20} {total_equity:>20} \n")
    s3 =(f"{'ABOOK':<10} {abook_balance:>20} {abook_equity:>20} \n")
    s4 =(f"{'BBOOK':<10} {bbook_balance:>20} {bbook_equity:>20} \n")

    summary = s1 + s2 + s3 + s4



    if caption:
        summary = f"*{caption}*\n{summary}"


    bot.send_message(chat_id=CHAT_ID, text=summary, parse_mode='Markdown')

# send_summary_to_telegram(df, df_2, caption='#PNL&Equity📊')

In [77]:
len('Период               АБУК + ББУК часть без партнёров         ')

61